In [8]:
import pandas as pd
from sodapy import Socrata
import sqlalchemy as sa
from dotenv import load_dotenv
import os
load_dotenv()

server = os.getenv('mshn_server_address')
un =os.getenv('mshn_server_uid')
pw = os.getenv('mshn_server_pw')

In [6]:
# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("chronicdata.cdc.gov", None)

# cwsq-ngmh is the tract data source, returned as JSON from API, and converted to Python list of
# dictionaries by sodapy.
results = client.get("cwsq-ngmh", limit=2125561)

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)

In [7]:
# Select variables and columns of interest
key_vars = [
'Physical health not good for >=14 days among adults aged >=18 years',
'Fair or poor self-rated health status among adults aged >=18 years',
'Current smoking among adults aged >=18 years',
'Obesity among adults aged >=18 years',
'Diagnosed diabetes among adults aged >=18 years',
'Depression among adults aged >=18 years',
'Mental health not good for >=14 days among adults aged >=18 years',
'Stroke among adults aged >=18 years']

column_names = ['year','datasource','locationname','short_question_text','measure','data_value']

# Select data for the latest year
latest_year = results_df['year'].astype('int').max().astype('str')

# Apply filters and formats
df = (results_df[(results_df['year'] == latest_year) & (results_df['measure'].isin(key_vars))]
.loc[:,column_names]
.rename(columns ={
    'locationname':'tract', 
    'datasource':'source',
    'data_value':'value',
    'short_question_text':'var_short_name',
    'measure':'var_name'})
.assign(variable = lambda df: df.var_name)
.reindex(columns = ['source','year','tract','var_short_name','value','var_name','variable'])
.reset_index(drop=True)
.assign(state_fips = lambda df: df["tract"].str[0:2])
)



In [ ]:
# Filter for state of MI
df=df[df.state_fips=='26']
   
df.drop('state_fips',axis = 1, inplace = True)

In [11]:
# Export to server

conn_string = "mssql+pyodbc://"+un+":"+pw+"@"+server+":1433/"+db+"?driver=ODBC+Driver+17+for+SQL+Server"

ms_sql_engine = sa.create_engine(conn_string,
                           fast_executemany = True)

df.to_sql('michigan_tract_att',con = ms_sql_engine, index = False, if_exists= 'append')

-1